Importing dependencies


In [49]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
import PIL as pillow
from PIL import Image, ImageOps
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout


In [50]:
# gpus = tf.config.experimental.list_physical_devices('GPU')

In [51]:
# print(len(gpus))

In [52]:
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)

Verifying image format

In [53]:
f = open('train_labels.txt', 'r')
dir = 'data/'
train_data = f.read()
train_data = train_data.split('\n')
train_labels = []
train_images = []
for line in train_data[1:]:
    line = line.split(',')
    if isinstance(line,list) and line[0]:
        x = line[0]
        img = pillow.Image.open(dir + x + '.png')
        img = ImageOps.grayscale(img)
        img = np.array(img)
        img = img / 255.0
        train_images.append(img)
        y = int(line[1])  
        train_labels.append(y)

In [54]:
f = open('validation_labels.txt', 'r')
dir = 'data/'
val_data = f.read()
val_data = val_data.split('\n')
val_labels = []
val_images = []
for line in val_data[1:]:
    line = line.split(',')
    if isinstance(line,list) and line[0]:
        x = line[0]
        img = pillow.Image.open(dir + x + '.png')
        img = ImageOps.grayscale(img)
        img = np.array(img)
        img = img / 255.0
        val_images.append(img)
        y = int(line[1])  
        val_labels.append(y)

In [55]:
train_images = np.array(train_images)
val_images = np.array(val_images)
train_labels = np.array(train_labels)
val_labels = np.array(val_labels)

In [56]:
model = Sequential()
# model.add(Conv2D(128, (3,3), 1, activation='relu', input_shape=(224,224,1)))
# model.add(MaxPooling2D(2,2))
# model.add(Dropout(0.4))
model.add(Conv2D(64, (3,3), 1, activation='relu', input_shape=(224,224,1)))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [57]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [58]:
cnn = model.fit(train_images, train_labels, epochs=1, batch_size=64, validation_data=(val_images, val_labels))

 94/235 [===========>..................] - ETA: 50:41 - loss: 0.3987 - accuracy: 0.8442

In [ ]:
f = open('sample_submission.txt', 'r')
dir = 'data/'
submission_data = f.read()
submission_data = submission_data.split('\n')
sub = open('submission.csv', 'w')
sub.write('id,class\n')

for line in submission_data[1:]:
    line = line.split(',')
    if isinstance(line,list) and line[0]:
        x = line[0]
        img = pillow.Image.open(dir + x + '.png')
        img = ImageOps.grayscale(img)
        img = np.array(img)
        img = img / 255.0
        res = model.predict(np.array([img]))[0][0]
        if res > 0.5:
            res = 1
        else:
            res = 0
        sub.write(x + ',' + str(res) + '\n')

sub.close()

1/1 [==============================] - 0s 56ms/step
